In [9]:
import pandas as pd
from sqlalchemy import create_engine

## Clean Airport Data and Push to Postgres

In [4]:
#read csv file
csv_file = "resources/airport_codes.csv"
airport_name_df = pd.read_csv(csv_file)

#rename columns to match postgres tables
airport_name_df.columns =["airport_code", "airport_name"]
airport_name_df.head()

,airport_code,airport_name
0,ATL,Hartsfield-Jackson Atlanta
1,BWI,Baltimore-Washington
2,BOS,Boston Logan
3,CLT,Charlotte Douglas
4,MDW,Chicago Midway


In [6]:
#push to postgres

#pip install psycopg2 
# rds_connection_string = "<insert user name>:<insert password>@localhost:5432/<insert database>"
rds_connection_string = "postgres:postgres@localhost:5432/airline_etl"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

# Use pandas to load csv converted DataFrame into database
airport_name_df.to_sql(name='airport', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "airport_pkey"
DETAIL:  Key (airport_code)=(ATL) already exists.

[SQL: INSERT INTO airport (airport_code, airport_name) VALUES (%(airport_code)s, %(airport_name)s)]
[parameters: ({'airport_code': 'ATL', 'airport_name': 'Hartsfield-Jackson Atlanta'}, {'airport_code': 'BWI', 'airport_name': 'Baltimore-Washington'}, {'airport_code': 'BOS', 'airport_name': 'Boston Logan'}, {'airport_code': 'CLT', 'airport_name': 'Charlotte Douglas'}, {'airport_code': 'MDW', 'airport_name': 'Chicago Midway'}, {'airport_code': 'ORD', 'airport_name': "Chicago O'Hare"}, {'airport_code': 'DAL', 'airport_name': 'Dallas Love Field'}, {'airport_code': 'DFW', 'airport_name': 'Dallas/Fort-Worth'}  ... displaying 10 of 30 total bound parameter sets ...  {'airport_code': 'DCA', 'airport_name': 'Ronald Reagan Washington'}, {'airport_code': 'IAD', 'airport_name': 'Washington Dulles'})]
(Background on this error at: http://s

## Clean Airline Data and Push to Postgres

In [7]:
#read csv file
csv_file = "resources/airline_codes.csv"
airline_df = pd.read_csv(csv_file)

#renmae columns to match postgres tables
airline_df.columns = ["airline_code", "airline_name"]

airline_df.head()

# Use pandas to load csv converted DataFrame into database
airline_df.to_sql(name='airline', con=engine, if_exists='append', index=False)

In [8]:
airline_df.head()



,airline_code,airline_name
0,AS,Alaska Airlines
1,G4,Allegient Air
2,AA,American Airlines
3,5Y,Atlas Airlines
4,DL,Delta Airlines


## Clean 2018 flight data

In [50]:
#This file is too big to upload to github.  Please download this file from https://www.kaggle.com/zernach/2018-airplane-flights
#this file is not included in the resources folder.
csv_file = "Cleaned_2018_Flights.csv"
airline_2018_df = pd.read_csv(csv_file)

#if "Cleaned_2018_Flights.csv" takes too long to load in it's entirety, uncomment line below to load the first 10K rows.
#airline_2018_df = airline_2018_df.iloc[0:10000,:]
airline_2018_df.head()

,Unnamed: 0,ItinID,MktID,MktCoupons,Quarter,Origin,OriginWac,Dest,DestWac,Miles,ContiguousUSA,NumTicketsOrdered,AirlineCompany,PricePerTicket
0,0,20181767585,2018176758501,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,672.87
1,1,20181767586,2018176758601,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,367.68
2,2,20181767587,2018176758701,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,417.94
3,3,20181767636,2018176763601,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,247.10
4,4,20181767637,2018176763701,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,276.35


In [52]:
cleaned_airline_2018_df = airline_2018_df[['Origin', 'Dest', 'Miles', 'AirlineCompany','PricePerTicket' ]]
cleaned_airline_2018_df.head()

,Origin,Dest,Miles,AirlineCompany,PricePerTicket
0,PHL,LAX,2402.0,AA,672.87
1,PHL,LAX,2402.0,AA,367.68
2,PHL,LAX,2402.0,AA,417.94
3,PHL,LAX,2402.0,AA,247.10
4,PHL,LAX,2402.0,AA,276.35


In [53]:
#rename columns
cleaned_airline_2018_df.columns =["origin_airport_code", "dest_airport_code", "miles", "airline_code",  "price" ]
cleaned_airline_2018_df.head()

,origin_airport_code,dest_airport_code,miles,airline_code,price
0,PHL,LAX,2402.0,AA,672.87
1,PHL,LAX,2402.0,AA,367.68
2,PHL,LAX,2402.0,AA,417.94
3,PHL,LAX,2402.0,AA,247.10
4,PHL,LAX,2402.0,AA,276.35


In [56]:
# Use pandas to load csv converted DataFrame into database
cleaned_airline_2018_df.to_sql(name='flight_2018', con=engine, if_exists='append', index=False)

In [57]:
pd.read_sql_query('select * from flight_2018', con=engine).head()

,origin_airport_code,dest_airport_code,miles,airline_code,price
0,PHL,LAX,2402,AA,672.87
1,PHL,LAX,2402,AA,367.68
2,PHL,LAX,2402,AA,417.94
3,PHL,LAX,2402,AA,247.10
4,PHL,LAX,2402,AA,276.35
